In [ ]:
import time
import sys
import traceback
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException

In [ ]:
URL = "https://iranopasmigirim.com/fa"
# Timeout settings
PAGE_LOAD_TIMEOUT = 2
ELEMENT_TIMEOUT = 2
POST_CLICK_WAIT = 2  # wait after clicking for confirmation text

In [ ]:
def prepare_driver(headless=False, lang="fa"):
    opts = webdriver.ChromeOptions()
    if headless:
        # headless can be detected by some servers; prefer visible while testing
        opts.add_argument("--headless=new")
    opts.add_argument(f"--lang={lang}")
    opts.add_argument("--start-maximized")
    # do not add automation-detection flags here
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    driver.set_page_load_timeout(PAGE_LOAD_TIMEOUT)
    return driver

In [ ]:
def safe_click(element, driver):
    """Try different click strategies for reliability."""
    try:
        element.click()
        return True
    except ElementClickInterceptedException:
        try:
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", element)
            time.sleep(0.3)
            element.click()
            return True
        except Exception:
            pass
    except Exception:
        pass
    # fallback: use ActionChains
    try:
        ActionChains(driver).move_to_element(element).click(element).perform()
        return True
    except Exception:
        return False

In [ ]:
def close_cookie_banner_if_present(driver, wait):
    """Attempt to close common overlays / cookie banners that might block the button."""
    # Try a handful of likely selectors / button texts in Farsi/English
    selectors = [
        (By.XPATH, "//button[contains(normalize-space(.), 'قبول')]"),
        (By.XPATH, "//button[contains(normalize-space(.), 'بستن')]"),
        (By.XPATH, "//button[contains(normalize-space(.), 'Accept')]"),
        (By.XPATH, "//button[contains(normalize-space(.), 'Close')]"),
        (By.CSS_SELECTOR, "button[aria-label*='close']"),
        (By.CSS_SELECTOR, ".cookie, .cookie-consent, .cookie-banner"),
    ]
    for by, sel in selectors:
        try:
            el = wait.until(EC.element_to_be_clickable((by, sel)))
            try:
                el.click()
                time.sleep(0.4)
                return True
            except Exception:
                try:
                    driver.execute_script("arguments[0].click();", el)
                    time.sleep(0.4)
                    return True
                except Exception:
                    continue
        except TimeoutException:
            continue
    return False

In [ ]:
def find_sign_button(driver, wait):
    """Try multiple strategies to locate the 'ثبت امضای ناشناس' button."""
    xpath_candidates = [
        "//button[contains(normalize-space(.), 'ثبت امضای ناشناس')]",       # exact phrase
        "//button[contains(normalize-space(.),'امضا')]",                    # shorter
        "//*[contains(normalize-space(.), 'ثبت امضای ناشناس')]",            # any element
        "//*[contains(normalize-space(.), 'امضا') and (name()='button' or @role='button')]" 
    ]
    for xp in xpath_candidates:
        try:
            el = wait.until(EC.element_to_be_clickable((By.XPATH, xp)))
            return el
        except TimeoutException:
            continue

    # CSS heuristics fallback (from site bundle: many buttons use 'group' and px/py classes)
    css_candidates = [
        "button.group", 
        "button[class*='px-12'][class*='py-5']",
        "div[role='button'] button",
        "a[href*='join-campaign']"
    ]
    for sel in css_candidates:
        try:
            el = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, sel)))
            return el
        except TimeoutException:
            continue

    # last resort: search all clickable buttons and return first that contains 'امضا' text
    try:
        buttons = driver.find_elements(By.TAG_NAME, "button")
        for b in buttons:
            try:
                txt = b.text or ""
                if "امضا" in txt or "ثبت" in txt:
                    return b
            except Exception:
                continue
    except Exception:
        pass

    return None
    

In [ ]:
def wait_for_confirmation(driver, wait, timeout=POST_CLICK_WAIT):
    """
    Wait for known success messages or button state changes.
    Returns (True/False, message)
    """
    # phrases to look for
    phrases = [
        "امضای شما با موفقیت ثبت شد",
        "امضا شد",
        "Signed ✓",
        "قبلاً امضا شده",
        "Already Signed"
    ]
    # build xpath to match any phrase
    xpath_union = " | ".join([f"//*[contains(normalize-space(.), '{p}')]" for p in phrases])
    try:
        el = WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.XPATH, xpath_union)))
        text = el.text.strip()
        return True, text
    except TimeoutException:
        # fallback: check if button text changed to include Signed or similar
        try:
            btns = driver.find_elements(By.XPATH, "//button")
            for b in btns:
                try:
                    t = b.text.strip()
                    if any(x in t for x in ("Signed", "امضا شد", "قبلاً امضا شده")):
                        return True, t
                except Exception:
                    continue
        except Exception:
            pass
    return False, "No confirmation text detected within timeout."

In [ ]:
def sign_once_with_selenium(headless=False, inject_cookies=None, screenshot_on_error="error_screenshot.png"):
    driver = prepare_driver(headless=headless)
    wait = WebDriverWait(driver, ELEMENT_TIMEOUT)
    try:
        driver.get(URL)
        time.sleep(1.0)  # give some dynamic content time to load

        # Optional: inject cookies (dictionary) before interacting, if user provided a legit session cookie
        if inject_cookies:
            # Must be on same domain to set cookies
            for k, v in inject_cookies.items():
                try:
                    driver.add_cookie({"name": k, "value": v, "path": "/", "domain": "iranopasmigirim.com"})
                except Exception:
                    # if adding fails, ignore; cookies sometimes require secure flag / other attributes
                    pass
            # reload so cookies take effect
            driver.get(URL)
            time.sleep(0.6)

        # Try to close cookie banner or overlays
        close_cookie_banner_if_present(driver, wait)

        # Find the sign button
        sign_btn = find_sign_button(driver, wait)
        if not sign_btn:
            print("Could not locate the sign button. Inspect the page HTML and update selectors.")
            # take a screenshot to help debugging
            driver.save_screenshot(screenshot_on_error)
            print(f"Saved screenshot: {screenshot_on_error}")
            return False

        # scroll & click safely
        driver.execute_script("arguments[0].scrollIntoView({block:'center', inline:'center'});", sign_btn)
        time.sleep(0.4)
        clicked = safe_click(sign_btn, driver)
        if not clicked:
            print("Failed to click the sign button.")
            driver.save_screenshot(screenshot_on_error)
            return False

        print("Clicked the sign button; waiting for confirmation...")

        ok, msg = wait_for_confirmation(driver, wait, timeout=POST_CLICK_WAIT)
        if ok:
            print("Confirmation detected:", msg)
        else:
            print("No clear confirmation detected:", msg)
            # save screenshot for inspection
            driver.save_screenshot(screenshot_on_error)
            print(f"Saved screenshot: {screenshot_on_error}")

        return ok

    except Exception as e:
        print("Unhandled error during automation:", str(e))
        traceback.print_exc()
        try:
            driver.save_screenshot(screenshot_on_error)
            print(f"Saved screenshot: {screenshot_on_error}")
        except Exception:
            pass
        return False
    finally:
        time.sleep(1.0)
        driver.quit()

In [ ]:
success = sign_once_with_selenium(headless=False, inject_cookies=None)